# 参考

http://mayo.hatenablog.com/entry/2014/07/11/111432

Python3.5でurllibに変更があったので、その部分を修正

# 準備

BeautifulSoupのインストール
> `pip install beatifulsoup4`

Mecab のインストール
> `sudo apt-get install libmecab-dev`

> `sudo apt-get install mecab mecab-ipadic-utf8`

> `pip install mecab-python3`

# Web文書のクロール

リンク先のURLを取得する

In [9]:
# coding:utf-8

import json
import urllib.request
from urllib.parse import urljoin
from bs4 import BeautifulSoup

def crawl(pages, depth=2):
    setpages = set()

    for i in range(depth):
        newpages = set()
        for page in pages:
            try:
                c = urllib.request.urlopen(page)
            except:
                print("Could not open %s" % page)
                continue
            soup = BeautifulSoup(c.read(), 'lxml')
            setpages.add(page)

            links = soup('a')
            for link in links:
                if ('href' in dict(link.attrs)):
                    url = urljoin(page, link['href'])
                    if url.find("'") != -1: continue
                    url = url.split('#')[0]
                    if url[0:4] == 'http' and not url in setpages:
                        newpages.add(url)
        pages = newpages

    return list(setpages)

In [10]:
urls = ["http://news.yahoo.co.jp/",]
pages = crawl(urls)

In [11]:
urls

['http://news.yahoo.co.jp/']

In [12]:
len(pages)

69

In [13]:
pages[:10]

['http://bylines.news.yahoo.co.jp/suzukiyuji/20160828-00061597/',
 'http://docs.yahoo.co.jp/info/mediastatement/',
 'http://news.yahoo.co.jp/theme/094891d4e28f66e92267/',
 'http://zasshi.news.yahoo.co.jp/article?a=20160829-00008005-jprime-ent',
 'http://news.yahoo.co.jp/zasshi',
 'http://news.yahoo.co.jp/theme/07d30c5ff19bef528698/',
 'http://bylines.news.yahoo.co.jp/kohyoungki/20160828-00061585/',
 'http://news.yahoo.co.jp/pickup/6212690',
 'http://headlines.yahoo.co.jp/hl?a=20160828-00000069-mai-soci',
 'http://headlines.yahoo.co.jp/hl?a=20160829-00000049-dal-ent']

In [14]:
with open("./urls.json", "w") as f:
    json.dump(pages, f)

# リンク先のページ内の文章を取得

In [15]:
# coding: utf-8

import sys
import json
import MeCab
import urllib.request
from collections import defaultdict
from operator import itemgetter
from bs4 import BeautifulSoup, NavigableString, Comment, Declaration


class HTMLParser():

    def get(self, url):
        try:
            c = urllib.request.urlopen(url)
        except:
            print("Could not open %s" % url)
            return ""

        soup = BeautifulSoup(c.read(), 'lxml')
        text = '\n'.join(self.__getNavigableStrings(soup))
        return text

    def __getNavigableStrings(self, soup):
        if isinstance(soup, NavigableString):
            if type(soup) not in (Comment, Declaration) and soup.strip():
                yield soup
        elif soup.name not in ('script', 'style'):
            for c in soup.contents:
                for g in self.__getNavigableStrings(c):
                    yield g

In [16]:
with open("urls.json", "r") as f:
    urls = json.load(f)

print("Count of urls : " + str(len(urls)))
STOP_WORD = " 。 、 「 」 （ ） ? ？ ： ， , ． ! ！ # $ % & ' ( ) = ~ | ` { } * + ? _ > [ ] @ : ; / . ¥ ^ 【 】 ￥ ＿ ／ 『 』 ＞ ？ ＿ ＊ ＋ ｀ ｜ 〜 ＊ ＋ ＞ ？ ＃ ” ＃ ＄ ％ ＆ ’ \" ・".split()

hp = HTMLParser()
tagger = MeCab.Tagger("-O wakati")
wakati_text = []

Count of urls : 69


## Mecabで分かち書き

In [23]:
for url in urls:
    text = hp.get(url)
    wakati_raw = tagger.parse(text)
    wakati_formalize = []
    for row in wakati_raw.split('\n'):
        row = row.rstrip().lower()
        for sw in STOP_WORD:
            row = row.replace(sw, '')
        wakati_formalize.append(row)
    wakati_text.append(' '.join(wakati_formalize))

with open("wakati_text.txt", 'w') as f:
    f.write('\n'.join(wakati_text))

In [24]:
wakati_text[:100]

['html  サザエさん  急落 は “ 脱 テレビ  時代 の 予兆  鈴木 祐司  - 個人 - yahoo  ニュース ニュース ニュース トピックス テーマ 意識 調査 全国 紙 パック 条件 指定 ユーザー ページ 購読 一覧 設定 トップ 速報 写真 映像 雑誌 個人 ビジネス 特集 意識 調査 ランキング 新着 記事 一覧 国内 国際 経済 エン タメ スポーツ it  科学 ライフ オーサー 一覧 現在 javascript が 無効 に なっ て い ます  yahoo  ニュース の すべて の 機能 を 利用 する ため に は  javascript の 設定 を 有効 に し て ください  javascript の 設定 を 変更 する 方法 は こちら   サザエさん  急落 は “ 脱 テレビ  時代 の 予兆  鈴木 祐司 \xa0  \xa0 次世代 メディア 研究所 長  メディア アナリスト  津田塾大学 研究 員 2016 年 8 月 28 日 17 時 1 分 配信 ツイート シェア ブック マーク フジテレビ の ホームページ より 人気 アニメ  サザエさん  の 視聴 率 が 急落 し て いる  ウィキペディア に は  平均 視聴 率 は 1989 年 1 月 15 日 から 2008 年 11 月 時点 で 22  3   平均 最高 視聴 率 は 1979 年 9 月 16 日 放送 の 39  4   関東 地区   アニメ 全体 歴代 高 視聴 率 ２ 位  と 紹介 さ れ て いる  最高 が 40  近く で  ８ 年 前 まで は 平均 で 20  も 獲る オバケ 番組 だっ た の で ある  筆者 に も 思い入れ の ある 番組 だ  ｎｈｋ の 編成 に い た 時代  こんな 分析 を し た   もし  サザエさん  が ｎｈｋ 総合 で 放送 さ れ て い たら  週間 接触 率 は どれ だけ 上がる の か   週間 接触 率 と は  一 週間 で ５ 分 以上 その チャンネル を 見る 人 の 比率  2010 年 頃  ｎｈｋ 総合 は ６ 割 ほど に 留まっ て い た  ところが  サザエさん  が １つ ある と  ５ ～ ６  も 比率 が 上がる と 出

# word2vecでベクトル化

In [25]:
import word2vec

In [26]:
word2vec.word2vec('wakati_text.txt', 'wakati_text.bin', size=300, verbose=True)

b'Starting training using file wakati_text.txt\n'b'Vocab size: 1995\n'b'Words in train file: 74944\n'

In [27]:
model = word2vec.load('wakati_text.bin')

In [29]:
indices, matrices = model.cosine('ブラジル')

In [32]:
model.generate_response(indices, matrices).tolist()

[('韓国', 0.9821542614090031),
 ('自殺', 0.9748796823449093),
 ('達成', 0.9747330620119778),
 ('開幕', 0.9744351205880897),
 ('小池', 0.9740251983736802),
 ('対策', 0.9726168790244487),
 ('―', 0.9723742432513167),
 ('新', 0.972155675542677),
 ('ひき逃げ', 0.9715508850870216),
 ('主演', 0.9710739019269972)]